# Dependencies

In [ ]:
import os
import cv2
import math
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

from keras.layers import *
from keras.applications import VGG16, ResNet50
from keras.optimizers import Adam, SGD
from datetime import datetime, timedelta
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical, plot_model
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Data Preparation

In [ ]:
# Local Path (Windows)
# dataset_path = r"E:\Dataset Skripsi\Dataset" # Folder created manually
# save_file_path = r"E:\Dataset Skripsi\File_Save" # Folder can be created automatically

In [ ]:
# Mount to Drive
from google.colab import drive
drive.mount('/content/drive')

# G-Drive Path
dataset_path = "/content/drive/MyDrive/Skripsi/Dataset" # Folder created manually
save_file_path = "/content/drive/MyDrive/Skripsi/File_Save" # Folder can be created automatically

In [ ]:
# count dataset total
def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

# Initialize the total amount for theft and normal classes
total_pencurian = 0
total_normal = 0

# Initialize the amount per category
kategori_pencurian = {'Gedung': 0, 'Ritel': 0, 'Motor': 0}
kategori_normal = {'Gedung': 0, 'Ritel': 0, 'Motor': 0}

# Loop for each category and subcategory
for category in ['Pencurian', 'Normal']:
    for subcategory in ['Gedung', 'Ritel', 'Motor']:
        directory = os.path.join(dataset_path, category, subcategory)
        file_count = count_files(directory)

        if category == 'Pencurian':
            total_pencurian += file_count
            kategori_pencurian[subcategory] += file_count
        else:
            total_normal += file_count
            kategori_normal[subcategory] += file_count

# Print Result
print(f"Total video kelas Pencurian\t: {total_pencurian}")
print(f"Total video kelas Normal\t: {total_normal}")
print("\nJumlah video Pencurian per-kategori:")
for subcategory, count in kategori_pencurian.items():
    print(f"  {subcategory}: {count}")
print("\nJumlah video Normal per-kategori:")
for subcategory, count in kategori_normal.items():
    print(f"  {subcategory}: {count}")

In [ ]:
IMAGE_HEIGHT = 120
IMAGE_WIDTH = 160
SEQUENCE_COUNT = 30 # frames count for capture video
BATCH_SIZE = 9

'''
  Batch_size is the number of data samples processed by the model in one
  training iteration. A larger batch_size uses more memory, but training
  process can be faster and more stable.
'''

# Preprocessing

In [ ]:
# function for split dataset to train, valiation, and test
def split_dataset(data_path, train_num=30, val_num=3, test_num=2):
  train_files = []
  val_files = []
  test_files = []

  for root, _, files in os.walk(data_path): # loop for all dataset file path
    for idx, file in enumerate(files): # loop for file video name
      video_path = os.path.join(root, file) # spesific dataset file path

      # get file path
      if idx < train_num:
        train_files.append(video_path)
      elif idx < train_num + val_num:
        val_files.append(video_path)
      elif idx < train_num + val_num + test_num:
        test_files.append(video_path)

      # else:
        # test_files.append(video_path)

    # Check for duplicates
    all_paths = train_files + val_files + test_files
    unique_paths = set(all_paths)

    # Check if there are duplicates
    if len(all_paths) != len(unique_paths):
      print("Warning: Duplicate paths found!")

  return train_files, val_files, test_files

In [ ]:
# function for labeling image
def labeling_img(video_path):
  file_name = os.path.basename(video_path) # get file video name
  file_name = file_name.split()[0] # take the first word

  if file_name == 'normal': # class: 0 for normal, 1 for pencurian
    label = 0
  else:
    label = 1

  return label

In [ ]:
# # function for get video frame to image (function_1: get video with interval in seconds)
# def get_video_frame(video_path, resize=(320, 240), frame_interval=1):
#   cap = cv2.VideoCapture(video_path)
#   fps = int(cap.get(cv2.CAP_PROP_FPS)) # get video fps
#   frame_count = 0
#   frames = []
#   labels = []

#   while True:
#     ret, frame = cap.read()

#     if not ret:
#       break

#     # Capture images from the video at intervals specified in seconds
#     if frame_count % (fps*frame_interval) == 0:
#       frame = cv2.resize(frame, resize)
#       '''
#         OpenCV uses the BGR (Blue, Green, Red) image color format,
#         so it needs to be converted to RGB (Red, Green, Blue) before modeling
#         process.
#       '''
#       frame = frame[:, :, [2, 1, 0]] # convert image color format
#       frames.append(frame)

#       # call function labeling image
#       label = labeling_img(video_path)
#       labels.append(label)

#     frame_count += 1

#   cap.release()
#   return frames, labels

In [ ]:
# function for get video frame to image (function_2: get video with count of frames)
def get_video_frame(video_path, resize=(320, 240), sequence_count=20):
  # get frames and labels list
  frames = []
  labels = []

  video_reader = cv2.VideoCapture(video_path)
  frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames = max(int(frames_count/sequence_count), 1) # set the value to 1 if the quotient is negative

  for frame_counter in range(sequence_count):
    # set the current frame position of the video.
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames)

    # read the video frame.
    success, frame = video_reader.read()

    # check if Video frame is not successfully read then break the loop
    if not success:
        break

    # convert image color format
    frame = frame[:, :, [2, 1, 0]]
    '''
      OpenCV uses the BGR (Blue, Green, Red) image color format,
      so it needs to be converted to RGB (Red, Green, Blue) before modeling
      process.
    '''

    # resize image
    frame = cv2.resize(frame, resize)

    # append the normalized frame into the frames list
    frames.append(frame)

    # call function labeling image
    # label = labeling_img(video_path)
    # labels.append(label)

  video_reader.release()
  # return frames, labels
  return frames

In [ ]:
# function for normalize pixel with range 0-1
def normalization_pixel(get_image):
  get_image = get_image.astype(np.float32) # convert integer to float
  normalized_image = get_image / 255.0 # normalize pixelz
  return normalized_image

In [ ]:
# function for show frames image (Optional)
def plot_images(images, main_title, rows=2, cols=4):
  # create a figure and a set of subplots
  fig, axes = plt.subplots(rows, cols, figsize=(12, 6))

  # Set main title
  fig.suptitle(main_title, fontsize=16)

  # iterate flattened array of axes
  for i, ax in enumerate(axes.flat):
    if i < len(images): # check if there are images to display
      ax.imshow(images[i], cmap='gray')
      ax.set_title(f'image_{i}')

    ax.axis('off') # turn off axis lines and labels

  plt.tight_layout() # adjust spacing between subplots
  plt.show()

In [ ]:
# # function for save image and labels (this function is optional)
# def save_files(get_image, get_labels, path_save, type_file, folder_name=None):
#   # path for save image
#   if folder_name is None:
#     path = os.path.join(path_save, type_file)
#   else:
#     path = os.path.join(path_save, folder_name, type_file)

#   os.makedirs(path, exist_ok=True) # create folder if not exist

#   # save image
#   for idx, img in enumerate(get_image):
#     img = img[:, :, [2, 1, 0]] # convert image color format

#     frame_name = f"frame_{idx}.jpg"
#     frame_path = os.path.join(path, frame_name)
#     cv2.imwrite(frame_path, img)

#   # save labels
#   path_labels = os.path.join(path, f"{type_file}.txt")
#   with open(path_labels, 'w') as f:
#     for label in get_labels:
#       f.write(str(label) + '\n')

In [ ]:
# function for save image and labels (this function is optional) | For Input TimeDistributed
def save_files(get_image, get_labels, path_save, type_file, folder_name=None):
  # folder path for save image
  if folder_name is None:
    path = os.path.join(path_save, type_file)
  else:
    path = os.path.join(path_save, folder_name, type_file)
  os.makedirs(path, exist_ok=True) # create folder if not exist

  for idx, (images, label) in enumerate(zip(get_image, get_labels)):
    frames_folder_path = os.path.join(path, f'frames_{idx}')
    os.makedirs(frames_folder_path, exist_ok=True) # create folder if not exist

    # save image
    for idx2, img in enumerate(images):
      img = img[:, :, [2, 1, 0]] # convert image color format

      frame_name = f"frame_{idx}_{idx2}.jpg"
      frame_path = os.path.join(frames_folder_path, frame_name)
      cv2.imwrite(frame_path, img)

    # save labels
    path_labels = os.path.join(path, f"{type_file}.txt")
    if os.path.exists(path_labels) and idx == 0:
      os.remove(path_labels)

    label_str = 1 if np.array_equal(label, [0, 1]) else 0
    with open(path_labels, 'a') as f:
      f.write(f'frames_{idx}: {label} {label_str}\n')

In [ ]:
# # split dataset path for train, validation, and test (function_1)
# train_path, val_path, test_path = split_dataset(dataset_path, train_num=5)

# # list image frame
# train_frames = []
# val_frames = []
# test_frames = []

# # list image labels
# train_labels = []
# val_labels = []
# test_labels = []

# interval = 2 # interval frame in second

# # get images and labels train
# for path in train_path:
#   frame, label = get_video_frame(video_path=path, frame_interval=interval)
#   train_frames.extend(frame)
#   train_labels.extend(label)

# # get images and labels validation
# for path in val_path:
#   frame, label = get_video_frame(video_path=path, frame_interval=interval)
#   val_frames.extend(frame)
#   val_labels.extend(label)

# # get images and labels test
# for path in test_path:
#   frame, label = get_video_frame(video_path=path, frame_interval=interval)
#   test_frames.extend(frame)
#   test_labels.extend(label)

# # convert list image to numpy array
# train_frames = np.array(train_frames)
# val_frames = np.array(val_frames)
# test_frames = np.array(test_frames)

# # show image shape
# print("Training shape images:", train_frames.shape)
# print("Valiation shape images:", val_frames.shape)
# print("Testing shape images:", test_frames.shape)

In [ ]:
# # split dataset path for train, validation, and test (function_2) | without list .extend()
# train_path, val_path, test_path = split_dataset(dataset_path, train_num=5)

# # ======================= Process The Images and Labels ========================
# # list image frame
# train_frames = []
# val_frames = []
# test_frames = []

# # list image labels
# train_labels = []
# val_labels = []
# test_labels = []

# # get images and labels train
# for path in train_path:
#   frame, label = get_video_frame(video_path=path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
#   train_frames.extend(frame)
#   train_labels.extend(label)

# # get images and labels validation
# for path in val_path:
#   frame, label = get_video_frame(video_path=path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
#   val_frames.extend(frame)
#   val_labels.extend(label)

# # get images and labels test
# for path in test_path:
#   frame, label = get_video_frame(video_path=path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
#   test_frames.extend(frame)
#   test_labels.extend(label)

# # convert list images to numpy array
# train_labels = np.array(train_labels)
# val_labels = np.array(val_labels)
# test_labels = np.array(test_labels)

# # convert list labels to numpy array
# train_frames = np.array(train_frames)
# val_frames = np.array(val_frames)
# test_frames = np.array(test_frames)

# # show image shape
# print("Training shape images:", train_frames.shape)
# print("Valiation shape images:", val_frames.shape)
# print("Testing shape images:", test_frames.shape)

# # show labels shape
# print("\nTraining shape labels:", train_labels.shape)
# print("Valiation shape labels:", val_labels.shape)
# print("Testing shape labels:", test_labels.shape)

In [ ]:
# split dataset path for train, validation, and test (function_2) | with list .append()
train_path, val_path, test_path = split_dataset(dataset_path, train_num=30, val_num=3, test_num=2)

# ======================= Process The Images and Labels ========================
# list image frame
train_frames = []
val_frames = []
test_frames = []

# list image labels
train_labels = []
val_labels = []
test_labels = []

# get images and labels train
for path in train_path:
  frame = get_video_frame(video_path=path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
  if len(frame) == SEQUENCE_COUNT: # check frame are the same as count of sequences
    train_frames.append(frame)

    # call function labeling image
    label = labeling_img(path)
    train_labels.append(label)

# get images and labels validation
for path in val_path:
  frame = get_video_frame(video_path=path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
  if len(frame) == SEQUENCE_COUNT: # check frame are the same as count of sequences
    val_frames.append(frame)

    # call function labeling image
    label = labeling_img(path)
    val_labels.append(label)

# get images and labels test
for path in test_path:
  frame = get_video_frame(video_path=path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
  if len(frame) == SEQUENCE_COUNT: # check frame are the same as count of sequences
    test_frames.append(frame)

    # call function labeling image
    label = labeling_img(path)
    test_labels.append(label)

# convert list images to numpy array
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)

# convert list labels to numpy array
train_frames = np.array(train_frames)
val_frames = np.array(val_frames)
test_frames = np.array(test_frames)

# show image shape
print("Training shape images:", train_frames.shape)
print("Valiation shape images:", val_frames.shape)
print("Testing shape images:", test_frames.shape)

# show labels shape
print("\nTraining shape labels:", train_labels.shape)
print("Valiation shape labels:", val_labels.shape)
print("Testing shape labels:", test_labels.shape)

# convert label to one-hot-encoding
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)
test_labels = to_categorical(test_labels)

# show one-hot-encoding labels shape
print("\nTraining shape one-hot-encoding labels:", train_labels.shape)
print("Valiation shape one-hot-encoding labels:", val_labels.shape)
print("Testing shape one-hot-encoding labels:", test_labels.shape)

In [ ]:
# show image original train (Optional)
ploting = train_frames.reshape(-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)[:8]
plot_images(ploting, 'Original Images')

In [ ]:
# show image original valid (Optional)
ploting = val_frames.reshape(-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)[:8]
plot_images(ploting, 'Original Images')

In [ ]:
# show image original test (Optional)
ploting = test_frames.reshape(-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)[:8]
plot_images(ploting, 'Original Images')

In [ ]:
# Save Original Image and Labels (Optional)
folderNames = 'Video Frames'
save_files(train_frames, train_labels, save_file_path, folder_name=folderNames, type_file='Train') # for train
save_files(val_frames, val_labels, save_file_path, folder_name=folderNames, type_file='Validation') # for validation
save_files(test_frames, test_labels, save_file_path, folder_name=folderNames, type_file='Test') # for test

In [ ]:
# Normalize pixel with range 0-1
train_frames_norm = normalization_pixel(train_frames)
val_frames = normalization_pixel(val_frames)
test_frames = normalization_pixel(test_frames)

# Print pixel range comparison
print(f"Range before normalization:\nmin: {np.min(train_frames)}\nmax: {np.max(train_frames)}")
print(f"\nRange after normalization:\nmin: {np.min(train_frames_norm)}\nmax: {np.max(train_frames_norm)}")

In [ ]:
# show image after normalization (Optional)
ploting = train_frames_norm.reshape(-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)[:8]
plot_images(ploting, 'Normalization Images')

## Augmentation (Tentatif)

In [ ]:
# # function for augmented training image (keras)
# def image_augmentation(get_images, get_labels, save_path=None, batch_size=32):
#   # path to save if not None
#   if save_path is not None:
#     save_path = os.path.join(save_path, 'Augmentation')
#     os.makedirs(save_path, exist_ok=True) # create folder if not exist

#   # initialize augmented image
#   datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     rescale=1./255)

#   # return the augmentation data
#   return datagen.flow(
#     x=get_images,
#     y=get_labels,
#     save_to_dir=save_path,
#     save_prefix='augmented',
#     batch_size=batch_size,
#     shuffle=False,
#     seed=42)

In [ ]:
# # augmented train image
# train_frames_aug = image_augmentation(reshape_train_frames,
#                                 reshape_train_labels,
#                                 batch_size=BATCH_SIZE)

# # =============== Check The Images and Labels After Augmentation ===============
# # get all augmented image
# aug_frames = []
# aug_labels = []

# # iterate images according to batch size
# for _ in range(len(reshape_train_frames) // BATCH_SIZE + 1):
#   batch_data, labels_data = train_frames_aug.next() # get image and labels in generator

#   # break if batch_data is none
#   if batch_data.shape[0] == 0:
#     break;

#   # save image array to list
#   aug_frames.extend(batch_data)
#   aug_labels.extend(labels_data)

# # convert list to numpy array
# aug_frames = np.array(aug_frames)

# # show image shape and range pixel
# print("Training augmentation shape images:", aug_frames.shape)
# print(f"\nAugmentation Pixel Image Range:\nmin: {np.min(batch_data)}\nmax: {np.max(batch_data)}")
# print(f'\nAugmented Labels Number: {len(aug_labels)}')

# # check the augmented label same or not as original labels
# if np.array_equal(aug_labels, train_labels):
#     print('The augmented labels and the original labels are the same')
# else:
#     print('The augmented labels and the original labels are different')

In [ ]:
# save the Augmentation Images and Labels
# save_files(np.uint8(aug_frames * 255), aug_labels, save_file_path, type_file='Augmentation')

In [ ]:
# show image augmentation
# plot_images(aug_frames[:8])

# Modeling

In [ ]:
# Function for get CNN filters
# def model_filters(layer):
#   filters, bias = layer.get_weights()
#   n_filters = filters.shape[-1]
#   n_channels = filters.shape[-2]

#   print(f'{layer.name}_bias:\n{bias}')
#   plt.figure(figsize=(n_channels*3, n_filters*3))
#   for i in range(n_filters):
#     for j in range(n_channels):
#       ax = plt.subplot(n_filters, n_channels, i*n_channels + j + 1)
#       ax.set_xticks([])
#       ax.set_yticks([])

#       plt.xlabel(f'depth_{j}') # set x label (filter depth)
#       if j == 0:
#         plt.ylabel(f'feture_map_{i}') # set y label (feture_map indeks)
#       plt.imshow(filters[:, :, j, i], cmap='gray')

#       # show text
#       # filter_value = filters[:, :, j]
#       # for x in range(filter_value.shape[0]):
#       #   for y in range(filter_value.shape[1]):
#       #     plt.text(y, x, filter_value[x, y, j], color='red', fontsize=6, ha='center', va='center')

#   plt.tight_layout()
#   plt.show()

In [ ]:
# Function for get CNN features_map
# def feature_map(img_path, model, idx_model):
#   layer = model.layers[idx_model]

#   # image preprocessing
#   image = cv2.imread(img_path)
#   image = image[:, :, [2, 1, 0]]
#   image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
#   image = normalization_pixel(image)

#   image = np.expand_dims(image, axis=0)
#   image = np.repeat(image, SEQUENCE_COUNT, axis=0)
#   image = image[np.newaxis, :]

#   layer_predict = Model(inputs=model.inputs, outputs=layer.output)
#   features_map = layer_predict.predict(image)

#   num_features = features_map.shape[-1]
#   num_rows = math.ceil(math.sqrt(num_features))
#   num_cols = math.ceil(num_features / num_rows)
#   plt.figure(figsize=(num_rows*3, num_cols*3))

#   for i in range(num_features):
#     plt.subplot(num_rows, num_cols, i+1)
#     plt.imshow(features_map[0, 0, :, :, i], cmap='gray')
#     plt.axis('off')
#     plt.title(f'feature_map_{i}')

#   plt.tight_layout()
#   plt.show()

In [ ]:
# Load pre-trained VGG16 model without top (fully connected layers)
model_name = 'VGG16_CNN-LSTM' # name for save model

# Load pre-trained VGG16 model without top (fully connected layers)
vgg_model = VGG16(weights='imagenet', include_top=False)

# Freeze the weights of the VGG16 model
for layer in vgg_model.layers:
    layer.trainable = False

# Create CNN-LSTM model
model = Sequential()

# Add VGG16 model to the sequential model
model.add(TimeDistributed(vgg_model, input_shape=(SEQUENCE_COUNT, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))

# Add Global Average Pooling
model.add(TimeDistributed(GlobalAveragePooling2D()))

# Add LSTM layers
# model.add(TimeDistributed(Flatten()))
model.add(LSTM(256))

# Add final dense layers
model.add(Dense(2, activation='sigmoid'))

model.summary()

In [ ]:
model_name = 'SelfBuild_CNN-LSTM' # name for save model

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                          input_shape = (SEQUENCE_COUNT, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))

model.add(TimeDistributed(MaxPooling2D((4, 4))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((4, 4))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
#model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Flatten()))
model.add(LSTM(32))
model.add(Dense(2, activation = 'sigmoid'))
model.summary()

In [ ]:
model_name = 'SelfBuild_CNN-LSTM' # name for save model

model = Sequential()

model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu'),
                          input_shape=(SEQUENCE_COUNT, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
# model.add(LSTM(128))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.50))

model.add(Dense(2, activation='sigmoid'))

model.summary()

In [ ]:
model_name = 'ResNet50' # name for save model

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

# Freeze ResNet layers
for layer in resnet_model.layers:
  layer.trainable = False

model = Sequential()
model.add(TimeDistributed(resnet_model, input_shape=(SEQUENCE_COUNT, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='sigmoid'))
model.summary()

### process architecture

In [ ]:
# create folder for save the model
current_datetime = datetime.now() + timedelta(hours=7) # current Indonesia date and time (GMT +7)
# current_datetime = datetime.now() # for windows

# set date format for main folder
date_format = '%y-%m-%d'
date = datetime.strftime(current_datetime, date_format)

# set time format for sub-folder
# time_format = '%H:%M'
time_format = '%H_%M' # for windows
time = datetime.strftime(current_datetime, time_format)

# create folder
model_save_folder = os.path.join(save_file_path, 'Model', date, f'{model_name}({time})') # folder path
os.makedirs(model_save_folder, exist_ok=True) # create folder if not exist

# Save and Show the Architecture Model Graph Image
save_file = os.path.join(model_save_folder, f'architecture.jpg') # file path
plot_model(model, to_file=save_file, show_shapes=True, show_layer_names=True)

In [ ]:
# Show CNN Filter (Optional)
# print('Model Indeks:')
# for idx, layer in enumerate(model.layers):
#   if 'time_distributed' not in layer.name or len(layer.get_weights()) != 2:
#     if len(layer.output.shape) == 5:
#       print(idx, layer.name, len(layer.get_weights()), '(Only For Show Feature_Map)')
#     continue
#   # if 'conv' not in layer.name:
#   #   continue

#   print(idx, layer.name, layer.get_weights()[0].shape)

In [ ]:
# Show CNN Features_Map (Optional)
# model_idx = 8

# # if len(model.layers[model_idx].get_weights()) == 2:
# #   model_filters(model.layers[model_idx])

# img_path = save_file_path + '/frame_0_0.jpg'
# feature_map(img_path, model, model_idx)

# Training

In [ ]:
# def plot_history(history):
#   metrics = list(history.history.keys()) # get dictionary key
#   num_plot = int(len(metrics) / 2) # for get only train key

#   _, axs = plt.subplots(1, num_plot, figsize=(30, 6)) # set canvas size

#   for idx in range(num_plot):
#     if metrics[idx] == 'f1_score': # f1-score return 2 values, because labels is one-hot-encoding
#       train_values = [np.mean(arr) for arr in history.history[metrics[idx]]] # get mean from array values
#       val_values = [np.mean(arr) for arr in history.history[f'val_{metrics[idx]}']] # get mean from array values
#     else:
#       train_values = history.history[metrics[idx]]
#       val_values = history.history[f'val_{metrics[idx]}']

#     # set up show plot data
#     axs[idx].plot(train_values, label=f'train_{metrics[idx]}')
#     axs[idx].plot(val_values, label=f'val_{metrics[idx]}')
#     axs[idx].set_title(f'{metrics[idx]} model')
#     axs[idx].set_xlabel('epoch')
#     axs[idx].set_ylabel(f'{metrics[idx]}')
#     axs[idx].legend()

#   # show ploting
#   plt.tight_layout()
#   plt.show()

In [ ]:
def Recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def Precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def F1_Score(y_true, y_pred):
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# Check device running on GPU
if tf.config.list_physical_devices('GPU'):
  print('GPU is available')
else:
  print('GPU is not available (USING CPU)')

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['Accuracy', 'Precision', 'Recall', F1_Score])

In [ ]:
# save the best training model
best_model_path = os.path.join(model_save_folder, 'best_model.keras')
checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# early stopping when there was no significant improvement in the training process
early_stopping = EarlyStopping(monitor='val_loss', patience=4, mode='auto')

# training the model (without augmentation)
model_history = model.fit(x = train_frames_norm,
                          y = train_labels,
                          validation_data= (val_frames, val_labels),
                          epochs = 100,
                          batch_size = BATCH_SIZE,
                          callbacks=[checkpoint, early_stopping],
                          shuffle = True)

# save the training model
model_save_path = os.path.join(model_save_folder, 'model.keras')
model.save(model_save_path)

In [ ]:
# show plot model history
metrics = list(model_history.history.keys()) # get dictionary key
num_plot = int(len(metrics) / 2) # for get only train key

_, axs = plt.subplots(1, num_plot, figsize=(30, 6)) # set canvas size

for idx in range(num_plot):
  # for colab:
  # if metrics[idx] == 'f1_score': # f1-score return 2 values, because labels is one-hot-encoding
  #   train_values = [np.mean(arr) for arr in model_history.history[metrics[idx]]] # get mean from array values
  #   val_values = [np.mean(arr) for arr in model_history.history[f'val_{metrics[idx]}']] # get mean from array values
  # else:
  #   train_values = model_history.history[metrics[idx]]
  #   val_values = model_history.history[f'val_{metrics[idx]}']

  train_values = model_history.history[metrics[idx]]
  val_values = model_history.history[f'val_{metrics[idx]}']

  # set up show plot data
  axs[idx].plot(train_values, label=f'train_{metrics[idx]}')
  axs[idx].plot(val_values, label=f'val_{metrics[idx]}')
  axs[idx].set_title(f'{metrics[idx]} model')
  axs[idx].set_xlabel('epoch')
  axs[idx].set_ylabel(f'{metrics[idx]}')
  axs[idx].legend()

# save the plot
save_img = os.path.join(model_save_folder, 'plotting_metrics.jpg')
plt.savefig(save_img)

# show ploting
plt.tight_layout()
plt.show()

# plot_history(model_history)

# Evaluation

In [ ]:
# evaluate the model
model_eval = load_model(model_save_path, custom_objects={'F1_Score': F1_Score})
result_model_eval = model_eval.evaluate(test_frames, test_labels)

In [ ]:
# evaluate the best model
best_model = load_model(best_model_path, custom_objects={'F1_Score': F1_Score})
result_best_model = best_model.evaluate(test_frames, test_labels)

In [ ]:
# Model Report
def report_metrics_value(model, metric_key, metric_name, idx, note):
  for key in metric_key[:metric_name]:
    if key == 'f1_score':
      value = np.mean(model.history[key][idx])
    else:
      value = model.history[key][idx]

    note.write('\ttrain_{}\t: {:.4}'.format(key, value))

  note.write('\n')
  for key in metric_key[metric_name:]:
    if key == 'val_f1_score':
      value = np.mean(model.history[key][idx])
    else:
      value = model.history[key][idx]

    note.write('\t{}\t: {:.4}'.format(key, value))

def report_eval(model, metric_key, note):
  for idx, value in enumerate(model):
    if isinstance(value, np.ndarray):
      value = np.mean(value)

    note.write('\t{}\t: {:.4}'.format(metric_key[idx], value))

# Save Files
save_report = os.path.join(model_save_folder, 'training_report.txt')
with open(save_report, 'w') as f:
  # Training report
  f.write(f'TRAINING REPORT📝\n\n')
  f.write(f'Image Size\t\t: {IMAGE_HEIGHT}(H) x {IMAGE_WIDTH}(W)\n')
  f.write(f'Total Train Data\t: {train_frames_norm.shape[0]}\nTotal Valid Data\t: {val_frames.shape[0]}\nTotal Test Data\t\t: {test_frames.shape[0]}\n')
  f.write(f'Frame Sequences\t\t: {SEQUENCE_COUNT}\n')
  f.write(f'Batch Size\t\t: {BATCH_SIZE}\n\n')

  f.write(f'Optimizer Name\t: {model.optimizer.get_config()["name"]}\n')
  f.write('learning rate\t: {:.5g}\n'.format(model.optimizer.get_config()['learning_rate']))

  config_str='\n\t'.join([f'{key}: {value}' for key, value in model.optimizer.get_config().items() if key != 'name' and key != 'learning_rate'])
  f.write(f'Optimizer Config:\n\t{config_str}\n\n')

  f.write(f'Loss\t: {model.loss}\n')
  f.write(f'Metrics\t: {metrics[1:num_plot]}\n\n')

  f.write(f'Total Epoch\t\t: {model_history.params["epochs"]}\n')
  f.write(f'Step Per-Epoch\t\t: {model_history.params["steps"]}\n')
  try:
    f.write(f'Epoch Early Stopping\t: {early_stopping.stopped_epoch}\n')
  except:
    f.write(f'Epoch Early Stopping\t: 0\n')

  idx = model_history.history['val_loss'].index(min(model_history.history['val_loss']))
  f.write(f'Best Model Epoch\t: {idx+1}\n\n')

  f.write('Training Model:\n')
  report_metrics_value(model_history, metrics, num_plot, idx=-1, note=f)

  f.write('\n\nTraining Best Model:\n')
  report_metrics_value(model_history, metrics, num_plot, idx=idx, note=f)

  # Evaluation Report
  f.write('\n\nEvaluation Model:\n')
  report_eval(result_model_eval, metrics, f)

  f.write('\n\nEvaluation Best Model:\n')
  report_eval(result_best_model, metrics, f)

In [ ]:
# get label predict and original
get_data = [train_frames, val_frames, test_frames]
get_labels = [train_labels, val_labels, test_labels]
titles = ['Train', 'Validation', 'Test']
name_confusion = 'Last Epoch Confusion Matrix'

fig, axes = plt.subplots(1, 3, figsize=(21, 7))
fig.suptitle(name_confusion, fontsize=16)

for idx, ax in enumerate(axes):
  model_predict = model_eval.predict(get_data[idx])
  label_predict = np.argmax(model_predict, axis=1)
  label_original = np.argmax(get_labels[idx], axis=1)
  conf_matrix = confusion_matrix(label_predict, label_original) # get confusion matrix

  sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title(titles[idx])
  ax.set_xticklabels(['Class 0', 'Class 1'])
  ax.set_yticklabels(['Class 0', 'Class 1'])

# save the plot
save_img = os.path.join(model_save_folder, name_confusion + '.jpg')
plt.savefig(save_img)

# show plot
plt.tight_layout()
plt.show()

In [ ]:
# get label predict and original
get_data = [train_frames, val_frames, test_frames]
get_labels = [train_labels, val_labels, test_labels]
titles = ['Train', 'Validation', 'Test']
name_confusion = 'Best Model Confusion Matrix'

fig, axes = plt.subplots(1, 3, figsize=(21, 7))
fig.suptitle(name_confusion, fontsize=16)

for idx, ax in enumerate(axes):
  model_predict = best_model.predict(get_data[idx])
  label_predict = np.argmax(model_predict, axis=1)
  label_original = np.argmax(get_labels[idx], axis=1)
  conf_matrix = confusion_matrix(label_predict, label_original) # get confusion matrix

  sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title(titles[idx])
  ax.set_xticklabels(['Class 0', 'Class 1'])
  ax.set_yticklabels(['Class 0', 'Class 1'])

# save the plot
save_img = os.path.join(model_save_folder, name_confusion + '.jpg')
plt.savefig(save_img)

# show plot
plt.tight_layout()
plt.show()

# Model Prediction

## Predict every 30 frame

In [ ]:
# def frame_preprocessing(image):
#   image = image[:, :, [2, 1, 0]]
#   image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
#   image = image.astype(np.float32)
#   image = image / 255.0
#   return image

# def model_predict(frames, model):
#   get_frame = [frame_preprocessing(image) for image in frames]
#   get_frame = np.array(get_frame)
#   get_frame = np.expand_dims(get_frame, axis=0)
#   return model.predict(get_frame)

# def show_predict(predict):
#   label = np.argmax(predict)
#   if label == 0:
#     text = 'Perilaku: Normal'
#   elif label == 1:
#     text = 'Perilaku: Pencurian'
#   else:
#     text = 'Detecting...'
#   return text

# def add_text(frame, text):
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     top_left_corner = (10, 30)
#     font_scale = 1
#     font_color = (0, 255, 0)
#     line_type = 2
#     cv2.putText(frame, text, top_left_corner, font, font_scale, font_color, line_type)

In [ ]:
# # folder path
# folder_path = "/content/drive/MyDrive/Skripsi/Data_Inference"

# # for root, _, files in os.walk(folder_path):
# #   for idx, get_file in enumerate(files):
# #     video_path = os.path.join(root, get_file)
# #     print(video_path)
# # ------------------------------------------------------------

# # video path
# # video_path = os.path.join(folder_path, 'Test_Video', 'Test_Video.mp4')
# # video_path = '/content/drive/MyDrive/Skripsi/Dataset/Normal/Ritel/normal (5).mp4'
# video_path = '/content/drive/MyDrive/Skripsi/Data_Inference/pencurian (3).mp4'

# # Load trained model
# model_path = os.path.join(save_file_path, 'Model', '24-05-09', 'SelfBuild_CNN-LSTM(15_45)', 'best_model.keras')

# cap = cv2.VideoCapture(video_path)
# model = load_model(model_path, custom_objects={'F1_Score': F1_Score})
# frames = []
# text = 'Detecting...'

# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))
# fps = int(cap.get(cv2.CAP_PROP_FPS))

# output_path = os.path.join(folder_path, 'Result', os.path.basename(video_path))
# frame2video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# while(True):
#   ret, frame = cap.read()

#   if not ret:
#     break;

#   frames.append(frame)
#   if len(frames) == SEQUENCE_COUNT:
#     prediction = model_predict(frames, model)
#     text = show_predict(prediction)
#     print(text)
#     frames = []

#   add_text(frame, text)
#   frame2video.write(frame)

# cap.release()
# frame2video.release()

## Predict only 30 frame

In [ ]:
# def frame_video(video_path, resize=(320, 240), sequence_count=20):
#   video_reader = cv2.VideoCapture(video_path) # get video
#   frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT)) # count of frame
#   skip_frames = max(frames_count//sequence_count, 1) # set the value to 1 if the quotient is negative
#   frames = []

#   for frame_counter in range(sequence_count):
#     video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames)
#     success, frame = video_reader.read()

#     # check if Video frame is not successfully read then break the loop
#     if not success:
#         break

#     frame = frame[:, :, [2, 1, 0]] # convert image format (BGR->RGB)
#     frame = cv2.resize(frame, resize) # resize image
#     frame = frame.astype(np.float32)
#     frame = frame / 255.0
#     frames.append(frame)

#   video_reader.release()
#   return frames

In [ ]:
# folder_path = "/content/drive/MyDrive/Skripsi/Data_Inference"
# video_path = folder_path + '/normal (2).mp4'
# model_path = os.path.join(save_file_path, 'Model', '24-05-16', 'SelfBuild_CNN-LSTM(14_50)', 'best_model.keras')

# frames = frame_video(video_path, resize=(IMAGE_WIDTH, IMAGE_HEIGHT), sequence_count=SEQUENCE_COUNT)
# frames = np.array(frames)
# frames = np.expand_dims(frames, axis=0)

# model = load_model(model_path, custom_objects={'F1_Score': F1_Score})
# predict = model.predict(frames)
# predict = np.argmax(predict)

# if predict == 0:
#   label = 'Normal'
# else:
#   label = 'Pencurian'

# print(label)

# Convert model format .keras to .H5

In [ ]:
# path_model = '/content/drive/MyDrive/Skripsi/File_Save/Model/best_model.keras'
# test_model = load_model(path_model, custom_objects={''F1_Score': F1_Score})
# test_model.save('/content/drive/MyDrive/Skripsi/File_Save/Model/test_model.h5')